In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import folium
from folium import plugins
import datetime as dt
import folium
import os
import sys
import gmplot
import webbrowser
from folium.plugins import FastMarkerCluster,HeatMap
from selenium import webdriver


%matplotlib inline 
plt.style.use('fivethirtyeight')
pd.set_option('display.max_columns', None)

# Import data

In [ ]:
df = pd.read_csv('data/incidenti_2007_2017.csv')

In [ ]:
df['DataOraIncidente'] = df['DataOraIncidente'].apply(lambda d: pd.to_datetime(d))
# df['DataOraIncidente_year'] = df['DataOraIncidente'].apply(lambda d: d.year)
# df['DataOraIncidente_month'] = df['DataOraIncidente'].apply(lambda d: d.month)
# df['DataOraIncidente_day'] = df['DataOraIncidente'].apply(lambda d: d.day)

In [ ]:
COORD_ROMA = df[~df['Latitudine'].isnull()][['Latitudine','Longitudine']].mean().values.tolist()
COORD_ROMA

# Incidenti con deceduti - punti singoli con descrizione

In [ ]:
m = folium.Map(
    location=[COORD_ROMA[0],COORD_ROMA[1]],
    zoom_start=12,
    tiles='cartodbpositron',
    #tiles='Stamen',
    #attr='Terrain',
    prefer_canvas=True,
    width='50%',
    height='50%')

columns = ['Latitudine',
           'Longitudine',
           'DataOraIncidente',
           'NaturaIncidente',
           'CondizioneAtmosferica',
           'FondoStradale',
           'Traffico']

df_temp = df[(~df['Latitudine'].isnull()) & (df['NUM_MORTI']>0)][columns]

for i in range(len(df_temp)):
    row = df_temp.iloc[i]
    popup = '<b>Data</b>: {}<br />'+\
            '<b>Orario</b>: {}<br />'+\
            '<b>Natura Incidente</b>: {}<br />'+\
            '<b>Condizione Atmosferica</b>: {}<br />'+\
            '<b>Fondo Stradale</b>: {}<br />'+\
            '<b>Traffico</b>: {}'
    popup = popup.format(str(row['DataOraIncidente'])[:10],
                         str(row['DataOraIncidente'])[11:],
                         row['NaturaIncidente'],
                         row['CondizioneAtmosferica'],
                         row['FondoStradale'],
                         row['Traffico'])
    folium.Circle(
        location=[row['Latitudine'],row['Longitudine']],
        radius=3,
        color='#0078ff80',
        popup=popup
    ).add_to(m)

#FastMarkerCluster(df_temp[['Latitudine','Longitudine']].values.tolist()).add_to(m)

m.save('maps/incidenti_deceduti.html')
webbrowser.open('file://' + os.path.realpath("maps/incidenti_deceduti.html"))

# Heatmap tutti gli incidenti

### Google (50000!!!)

In [ ]:
# latlong = df[~df['Latitudine'].isnull()][['Latitudine','Longitudine']].head(50000).values.tolist()

# gmap = gmplot.GoogleMapPlotter.from_geocode("Rome", zoom=11.8)
# gmap.heatmap([x[0] for x in latlong],
#              [x[1] for x in latlong])
# gmap.draw('maps/heatmap_tutti_google_50k.html')
# webbrowser.open('file://' + os.path.realpath('maps/heatmap_tutti_google_50k.html'))

### Folium

In [ ]:
m = folium.Map(
    location=[COORD_ROMA[0],COORD_ROMA[1]],
    zoom_start=10,
    tiles='cartodbpositron',
    width='50%',
    height='50%'
)

latlong = df[~df['Latitudine'].isnull()][['Latitudine','Longitudine']].head(50000).values.tolist()
HeatMap(latlong, radius=10).add_to(m)

path = 'maps/heatmap_tutti_folium.html'
m.save(path)
webbrowser.open('file://' + os.path.realpath(path))

In [ ]:
# for radius in [0.1, 1, 5, 10, 20]:
#     m = folium.Map(
#         location=[COORD_ROMA[0],COORD_ROMA[1]],
#         zoom_start=12,
#         tiles='cartodbpositron',
#         width='50%',
#         height='50%'
#     )
#     HeatMap(latlong,radius=radius).add_to(m)
#     path = 'maps/heatmap_tutti_folium_radius_{}.html'.format(radius)
#     m.save(path)

# Strade

In [ ]:
top_strade = df.groupby('Strada1')\
               .count()[['ID']]\
               .sort_values(by='ID', ascending=False)\
               .head(10).index.values
for strada in top_strade:
    m = folium.Map(
        location=[COORD_ROMA[0],COORD_ROMA[1]],
        zoom_start=10,
        tiles='cartodbpositron',
        width='50%',
        height='50%'
    )


    HeatMap(df[(~df['Latitudine'].isnull()) & ((df['Strada1']==strada) | (df['Strada2']==strada))][['Latitudine','Longitudine']].values.tolist(), 
            radius=10).add_to(m)

    m.save("maps/strade/{}.html".format(strada.replace(' ','_')))
    #webbrowser.open('file://' + os.path.realpath("maps/colombo.html"))

# Heatmap - evolution month by month

In [ ]:
df[~df['Latitudine'].isnull()].groupby(['DataOraIncidente_year','DataOraIncidente_month']).count()[['ID']].plot()

In [ ]:
def get_map():
    return folium.Map(
                        location=[COORD_ROMA[0],COORD_ROMA[1]],
                        zoom_start=10,
                        tiles='cartodbpositron',
                        width='100%',
                        height='100%'
                    )


for y in [2014,2015,2016]:
    for m in range(1,12+1):
        if (y==2014) and (m<4):
            pass
        else:
            mymap = get_map()
            HeatMap(df[(~df['Latitudine'].isnull()) & (df['DataOraIncidente_year']==y) & (df['DataOraIncidente_month']==m)][['Latitudine','Longitudine']].values.tolist(), 
            radius=10).add_to(mymap)
            mymap.save("maps/evolution/{}_{}.html".format(y,m))

In [ ]:
import os
import time
from selenium import webdriver

delay=5
fn='maps/evolution/2014_7.html'
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
#print(tmpurl)
#m.save(fn)

browser = webdriver.Firefox()
browser.get(tmpurl)
#Give the map tiles some time to load
time.sleep(delay)
browser.save_screenshot('maps/evolution/screenshots/test.png')
browser.quit()

*Commands to create the gif*. 

1)  mogrify -resize 640x480 *.png  
2)  convert -delay 50 -loop 0 *.png mygif.gif  


In [ ]:
#for y in [2014,2015,2016]:
for y in [2015,2016]:
    for m in range(1,12+1):
        if (y==2014) and (m<4):
            pass
        else:
            map_file = "maps/evolution/{}_{}.html".format(y,m)
            url = 'file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=map_file)
            print(url)
            browser = webdriver.Firefox()
            browser.get(url)
            time.sleep(delay)
            browser.save_screenshot('maps/evolution/screenshots/{}_{}.png'.format(y,m))
            browser.quit()

# Evolution - year by year

In [ ]:
def get_map():
    return folium.Map(
                        location=[COORD_ROMA[0],COORD_ROMA[1]],
                        zoom_start=10,
                        tiles='cartodbpositron',
                        width='100%',
                        height='100%'
                    )


for y in [2014,2015,2016]:
    mymap = get_map()
    HeatMap(df[(~df['Latitudine'].isnull()) & (df['DataOraIncidente_year']==y)][['Latitudine','Longitudine']].values.tolist(), 
    radius=10).add_to(mymap)
    mymap.save("maps/evolution/year/{}.html".format(y))

# Incidenti con pedoni - punti singoli

In [ ]:
m = folium.Map(
    location=[COORD_ROMA[0],COORD_ROMA[1]],
    zoom_start=12,
    tiles='cartodbpositron',
    #tiles='Stamen',
    #attr='Terrain',
    prefer_canvas=True,
    width='50%',
    height='50%')

columns = ['Latitudine',
           'Longitudine',
           'DataOraIncidente',
           #'NaturaIncidente',
           'CondizioneAtmosferica',
           'FondoStradale',
           'Traffico',
           'NUM_MORTI']

df_temp = df[(~df['Latitudine'].isnull()) & (df['NaturaIncidente']=='Investimento di pedone')][columns]
print('LEN:',len(df_temp))

for i in range(len(df_temp)):
    row = df_temp.iloc[i]
    popup = '<b>Data</b>: {}<br />'+\
            '<b>Orario</b>: {}<br />'+\
            '<b>Condizione Atmosferica</b>: {}<br />'+\
            '<b>Fondo Stradale</b>: {}<br />'+\
            '<b>Traffico</b>: {}'
    popup = popup.format(str(row['DataOraIncidente'])[:10],
                         str(row['DataOraIncidente'])[11:],
                         row['CondizioneAtmosferica'],
                         row['FondoStradale'],
                         row['Traffico'])
    folium.Circle(
        location=[row['Latitudine'],row['Longitudine']],
        radius=3,
        color = 'red' if row['NUM_MORTI']>0 else '#0078ff80',
        popup=popup
    ).add_to(m)
    
    if i%100==0:
        print(i,end=' ')

map_file = 'maps/incidenti_pedoni.html'
m.save(map_file)
webbrowser.open('file://' + os.path.realpath(map_file))

# Illuminazione insufficiente o inesistente

In [ ]:
m = folium.Map(
    location=[COORD_ROMA[0],COORD_ROMA[1]],
    zoom_start=12,
    tiles='cartodbpositron',
    #tiles='Stamen',
    #attr='Terrain',
    prefer_canvas=True,
    width='50%',
    height='50%')

columns = ['Latitudine',
           'Longitudine',
           'DataOraIncidente',
           #'NaturaIncidente',
           'CondizioneAtmosferica',
           'FondoStradale',
           'Traffico',
           'NUM_MORTI']

df_temp = df[(~df['Latitudine'].isnull()) & (df['Illuminazione'].isin(['Inesistente','Insufficiente']))][columns]
print('LEN:',len(df_temp))

for i in range(len(df_temp)):
    row = df_temp.iloc[i]
    popup = '<b>Data</b>: {}<br />'+\
            '<b>Orario</b>: {}<br />'+\
            '<b>Condizione Atmosferica</b>: {}<br />'+\
            '<b>Fondo Stradale</b>: {}<br />'+\
            '<b>Traffico</b>: {}'
    popup = popup.format(str(row['DataOraIncidente'])[:10],
                         str(row['DataOraIncidente'])[11:],
                         row['CondizioneAtmosferica'],
                         row['FondoStradale'],
                         row['Traffico'])
    folium.Circle(
        location=[row['Latitudine'],row['Longitudine']],
        radius=3,
        color = 'red' if row['NUM_MORTI']>0 else '#0078ff80',
        popup=popup
    ).add_to(m)
    
    if i%100==0:
        print(i,end=' ')

map_file = 'maps/incidenti_cattiva_illuminazione.html'
m.save(map_file)
webbrowser.open('file://' + os.path.realpath(map_file))

# bagnato vs asciutto

In [ ]:
df.groupby('FondoStradale')\
  .count()[['ID']]\
  .sort_values(by='ID', ascending=False)

In [ ]:
def get_map():
    return folium.Map(
                        location=[COORD_ROMA[0],COORD_ROMA[1]],
                        zoom_start=10,
                        tiles='cartodbpositron',
                        width='100%',
                        height='100%'
                    )


df_temp = df[(~df['Latitudine'].isnull())]

mymap = get_map()
df_bagnato = df_temp[df_temp['FondoStradale'].isin(['Bagnato (pioggia)','Bagnato (umidità in atto)'])]
#df_bagnato = df_bagnato.sample(len(df_asciutto))
HeatMap(df_bagnato[['Latitudine','Longitudine']].values.tolist(), 
        radius=10).add_to(mymap)
mymap.save("maps/bagnato_asciutto/bagnato.html")


mymap = get_map()
df_asciutto = df_temp[df_temp['FondoStradale']=='Asciutto']
df_asciutto = df_asciutto.sample(len(df_bagnato))
HeatMap(df_asciutto[['Latitudine','Longitudine']].values.tolist(), 
        radius=10).add_to(mymap)
mymap.save("maps/bagnato_asciutto/asciutto.html")

# Google maps

### Markers

In [ ]:
columns = ['Latitudine',
           'Longitudine',
           'DataOraIncidente',
           'NaturaIncidente',
           'CondizioneAtmosferica',
           'FondoStradale',
           'Traffico']

# CON DECEDUTI
df_temp = df[(~df['Latitudine'].isnull()) & (df['NUM_MORTI']>0)][columns]
gmap = gmplot.GoogleMapPlotter.from_geocode("Rome", zoom=11)
for i in range(len(df_temp)):
    row = df_temp.iloc[i]
    popup = 'Data: {}\\n'+\
            'Orario: {}\\n'+\
            'Natura Incidente: {}\\n'+\
            'Condizione Atmosferica: {}\\n'+\
            'Fondo Stradale: {}\\n'+\
            'Traffico: {}\\n'
    popup = popup.format(str(row['DataOraIncidente'])[:10],
                         str(row['DataOraIncidente'])[11:],
                         row['NaturaIncidente'],
                         row['CondizioneAtmosferica'],
                         row['FondoStradale'],
                         row['Traffico'])   
    gmap.marker(row['Latitudine'],row['Longitudine'], title=popup, color='red')
    

# #SENZA DECEDUTI    
# df_temp = df[(~df['Latitudine'].isnull()) & (df['NUM_MORTI']==0)][columns]
# gmap = gmplot.GoogleMapPlotter.from_geocode("Rome", zoom=11)
# for i in range(len(df_temp)):
#     row = df_temp.iloc[i]
#     popup = 'Data: {}\\n'+\
#             'Orario: {}\\n'+\
#             'Natura Incidente: {}\\n'+\
#             'Condizione Atmosferica: {}\\n'+\
#             'Fondo Stradale: {}\\n'+\
#             'Traffico: {}\\n'
#     popup = popup.format(str(row['DataOraIncidente'])[:10],
#                          str(row['DataOraIncidente'])[11:],
#                          row['NaturaIncidente'],
#                          row['CondizioneAtmosferica'],
#                          row['FondoStradale'],
#                          row['Traffico'])   
#     gmap.marker(row['Latitudine'],row['Longitudine'], title=popup, color='blue')

    
gmap.draw("maps/punti_singoli.html")
webbrowser.open('file://' + os.path.realpath("maps/punti_singoli.html"))

### Heatmap

In [ ]:
#df_temp = df[~df['Latitudine'].isnull()][columns]#.head(1000)
latlong = df[~df['Latitudine'].isnull()][['Latitudine','Longitudine']].head(50000).values.tolist()

gmap = gmplot.GoogleMapPlotter.from_geocode("Rome", zoom=11.8)
gmap.heatmap([x[0] for x in latlong],
             [x[1] for x in latlong])
gmap.draw("mymap.html")
webbrowser.open('file://' + os.path.realpath("mymap.html"))

# Strade

# Natura incidente